In [2]:
from numpy import array
from numpy import asarray
from numpy import zeros
import pandas as pd
import numpy as np

import re
import pickle 
#import mglearn
import time
# import gensim
import os
import collections
# import smart_open
import random
import string
import nltk

from nltk import Text
from nltk.tokenize import TweetTokenizer # doesn't split at apostrophes
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import word_tokenize  
from nltk.tokenize import sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from sklearn import decomposition
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

In [3]:
import preprocessor as p

In [4]:
import keras.backend as K
import keras.layers as layers
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from keras.callbacks import ModelCheckpoint
from keras.engine import Layer
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Embedding, Dense, Flatten, Input
from keras.layers import SpatialDropout1D, add, concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPool1D
from keras.models import Model
from keras.preprocessing import text, sequence


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [5]:
from hltc_preprocess import tweets
import tweepy
import pandas as pd

In [6]:
import glob

# 0. 같은 감정 데이터들 하나로 묶고 저장

anticipation은 할 필요 없음 

In [34]:
def data_load(file_path, emo):
    files = glob.glob(file_path+'tweet_data/{0}/*.csv'.format(emo))
    
    all_emo_data = []
    for file in files:     
        data = pd.read_csv(file, names=['text'], header=0)
        all_emo_data.append(data)
        
    concated_data = pd.concat(all_emo_data)
    concated_data.to_csv(file_path+"tweet_data/{0}/{0}_concat_data.csv".format(emo))
    print(emo,"concat data is saved")
    return concated_data

In [35]:
if __name__ == "__main__":
    emo = 'anticipation'
    file_path = 'C:/Users/Administrator/Anaconda3/envs/temp/EmotionEmbedding/emotion8_finalcode/data/'
    concated_data = data_load(file_path, emo)

anticipation concat data is saved


# 1. each emo concat data labeling // all emo concat

In [8]:
def concat_data_labeling(path, emo, emo_label_dic):
    file_emo = glob.glob(path)
    data = pd.read_csv(file_emo[0], header=0)
    data = data.reset_index(drop= True)
    data = data.dropna()
    label_num = emo_label_dic[emo]
    data['label'] = label_num
#     display(data.tail(10))
    
    return data

In [9]:
# function for cleaning the plots of the movies
def clean_text(text):
    text = text.lower()
    ## 특수기호제거 
    text = re.sub('[^A-Za-z ]', '',text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    #text = re.sub('\W', ' ', text)
    #text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [10]:
def tweet_preprocessing(data):
    emo_data_clean = data.copy()
    emo_data_clean['clean_text'] = emo_data_clean['text'].apply(tweets.clean_tweet)
    emo_data_clean['clean_text'] = emo_data_clean['clean_text'].apply(clean_text)
    emo_data_clean = emo_data_clean[['clean_text','label']]
    return emo_data_clean

In [11]:
if __name__ == "__main__":
    emo_list = ['anger','anticipation','disgust','fear','joy','trust','sadness','surprise']
    label_list = list(range(len(emo_list)))
    emo_label_dic = dict(zip(emo_list, label_list))
    
    all_emo_clean_data = []
    file_path = 'C:/Users/Administrator/Anaconda3/envs/temp/EmotionEmbedding/emotion8_finalcode/data/'
    
    for emo in emo_list:
        path = file_path+'tweet_data/{0}/{0}_concat_data.csv'.format(emo)
        data = concat_data_labeling(path, emo, emo_label_dic)
        
        emo_data_clean = tweet_preprocessing(data)
        print(emo, "data shape : ", emo_data_clean.shape)
        
        emo_data_clean.to_csv(file_path+"tweet_data/{0}/{0}_emo_clean_data3.csv".format(emo))
        all_emo_clean_data.append(emo_data_clean)
        
    concated_emo_clean_data = pd.concat(all_emo_clean_data)
    concated_emo_clean_data.to_csv(file_path+"tweet_data/all_emo_clean_data/all_emo_clean_data3.csv")

anger data shape :  (44488, 2)
anticipation data shape :  (8089, 2)
disgust data shape :  (8678, 2)
fear data shape :  (20012, 2)
joy data shape :  (22489, 2)
trust data shape :  (10697, 2)
sadness data shape :  (20462, 2)
surprise data shape :  (9786, 2)


In [12]:
result = pd.read_csv(file_path+"tweet_data/all_emo_clean_data/all_emo_clean_data3.csv")
result

,Unnamed: 0,clean_text,label
0,0,sorry didnt realize people had to hold your ha...,0
1,1,yesterday was so fucked upi aint with the bs t...,0
2,3,i hate that no matter how hard i try money con...,0
3,4,the young dudes on the game are very good but ...,0
4,5,ok heres a new one watching starrcade number i...,0
5,6,this morning i was woken up by the alarm from ...,0
6,7,my numberam meeting cancelled roar unleashfury...,0
7,8,its hard to study for a midterm when you reall...,0
8,10,hate when i forget to put my headphones back i...,0
9,15,there is a nice way to reprimand somebody peop...,0


# Tweet data label check 

In [2]:
import pandas as pd

In [4]:
# final label check 
file_path = 'C:/Users/Administrator/Anaconda3/envs/temp/EmotionEmbedding/emotion8_finalcode/data/'
final_data = pd.read_csv(file_path+"tweet_data/all_emo_clean_data/all_emo_clean_data3.csv")

In [6]:
final_data['label'].value_counts()

0    44488
4    22489
6    20462
3    20012
5    10697
7     9786
2     8678
1     8089
Name: label, dtype: int64

In [7]:
for i in range(8):
    display(final_data[final_data['label']==i].head())

,Unnamed: 0,clean_text,label
0,0,sorry didnt realize people had to hold your ha...,0
1,1,yesterday was so fucked upi aint with the bs t...,0
2,3,i hate that no matter how hard i try money con...,0
3,4,the young dudes on the game are very good but ...,0
4,5,ok heres a new one watching starrcade number i...,0


,Unnamed: 0,clean_text,label
44488,0,is it me or anyone else also noticed the weddi...,1
44489,3,how many of you out there use something like s...,1
44490,4,yarn the vampire diaries tvd excited curious...,1
44491,6,i wish there was some sort of scanner that wou...,1
44492,7,user does ur number eggs for males work for fe...,1


,Unnamed: 0,clean_text,label
52577,1,was considering booking up for switzerland est...,2
52578,2,i see where your money went and it wasnt to th...,2
52579,6,when the elevator smells like cigarettes ltlt ...,2
52580,7,if my roommate continues to leave his trash al...,2
52581,8,people whos north faces are completely coated ...,2


,Unnamed: 0,clean_text,label
61255,146,finding out that im going to have honors class...,3
61256,147,stressed to the max thanks to the audition i h...,3
61257,148,yay interview nervous,3
61258,151,user where are you youre number minutes late w...,3
61259,152,hope tomorrow goes well nervous,3


,Unnamed: 0,clean_text,label
81267,0,drama is finished not the best ending ever but...,4
81268,1,how in the hell is user girl album cover racis...,4
81269,3,epic day godbless everyone smile happy,4
81270,6,im sososo happy smile happydays,4
81271,9,this time next week i shall be touching down i...,4


,Unnamed: 0,clean_text,label
103756,1,awwwe smile relief alonelyplacetodie,5
103757,4,things are looking up yay relieved,5
103758,6,day number complete number on my systems test ...,5
103759,9,my bf has spent a week on phone to user post i...,5
103760,10,finally all caught up relieved,5


,Unnamed: 0,clean_text,label
114453,0,going to be an emotional day tomorrow sad,6
114454,5,user how come you didnt sign up it would of be...,6
114455,7,i need a haircut desperate,6
114456,8,im fucking getting sick sadtweet,6
114457,10,the sad moment when you find an old conversati...,6


,Unnamed: 0,clean_text,label
134915,0,just remembered i am on duty all weekend surpr...,7
134916,1,im actually getting a lot done today surprised,7
134917,2,today ive seen a dog that looked literally lik...,7
134918,4,how true is it that drinking too much water ca...,7
134919,9,whaat club lastnight was pumping catch yaz the...,7
